Cleaning merged league data from FBref:

In [1]:
# Import libraries:
import pandas as pd
import os

In [ ]:
# Global variables:
repo_dir = os.path.dirname(os.path.abspath(__file__))  # Directory of the script
merged_data_dir = os.path.join(repo_dir, "Merged Data")   # Path to the Merged Data folder
league_ids = ['ENG-Premier League', 'ESP-La Liga', 'FRA-Ligue 1', 'GER-Bundesliga', 'ITA-Serie A'] # For league-level iteration